# Attempt 2

This attempt is to use ViT from the huggingface project to finetune for classification. Currently bottlenecked by memory

In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = "0"
os.environ["WANDB_DISABLED"] = "true"

In [2]:
import numpy as np
import torch 
import torch.nn as nn
import torch.nn.functional as F
# import pytorch_lightning as pl
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import os
from sklearn.metrics import r2_score, mean_squared_error
from skimage.metrics import peak_signal_noise_ratio as psnr, structural_similarity as ssim
from collections import OrderedDict
from torchvision import transforms
from torchvision.datasets import ImageFolder
from PIL import Image
from transformers import ViTConfig, ViTForImageClassification, ViTImageProcessor, TrainingArguments, Trainer
from datasets import load_dataset
from transformers import DefaultDataCollator
import evaluate
from sklearn.metrics import accuracy_score, classification_report, roc_curve, auc, RocCurveDisplay

In [3]:
from datasets import load_dataset
dataset = load_dataset("csv", data_files="train.csv")

Using custom data configuration default-eb28a93c5c4b9d41
Reusing dataset csv (/home/201851018/.cache/huggingface/datasets/csv/default-eb28a93c5c4b9d41/0.0.0/51cce309a08df9c4d82ffd9363bbe090bf173197fc01a71b034e8594995a1a58)


  0%|          | 0/1 [00:00<?, ?it/s]

In [4]:
train_labels = dataset['train']['has_under_extrusion']

In [5]:
dataset

DatasetDict({
    train: Dataset({
        features: ['img_path', 'printer_id', 'print_id', 'has_under_extrusion'],
        num_rows: 81060
    })
})

In [6]:
def transform_images(examples):
    examples["pixel_values"] = [transforms.Resize((224, 224))(transforms.ToTensor()(Image.open("images/"+img).convert("RGB"))) for img in examples["img_path"]]
#     del examples["image"]
    del examples['printer_id']
    del examples['print_id']
    examples['label'] = examples['has_under_extrusion']
    del examples['has_under_extrusion']
    return examples


In [7]:
dataset = dataset.with_transform(transform_images)

Parameter 'transform'=<function transform_images at 0x7f52100dd3a0> of the transform datasets.arrow_dataset.Dataset.set_format couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


In [8]:
data_collator = DefaultDataCollator()


In [9]:
#vit_extractor =  ViTImageProcessor(resample="PIL.Image.BOX", do_normalize=True, size={"height":224, "width":224})
vit_extractor = ViTImageProcessor.from_pretrained('google/vit-base-patch16-224')
model = ViTForImageClassification.from_pretrained('google/vit-base-patch16-224', num_labels=2, ignore_mismatched_sizes=True)

Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([1000, 768]) in the checkpoint and torch.Size([2, 768]) in the model instantiated
- classifier.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([2]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
accuracy = evaluate.load("accuracy")


In [11]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [12]:
training_args = TrainingArguments(
    output_dir="vit_for_lenses",
    remove_unused_columns=False,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=1e-4,
    per_device_train_batch_size=16,
    gradient_accumulation_steps=4,
    per_device_eval_batch_size=16,
    num_train_epochs=4,
    warmup_ratio=0.1,
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    push_to_hub=False,
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [13]:
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset["train"],
#     eval_dataset=dataset["test"],
    tokenizer=vit_extractor,
    compute_metrics=compute_metrics,
)

In [14]:
trainer.train()


/home/201851018/.conda/envs/pytorchenv/lib/python3.9/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
/home/201851018/.conda/envs/pytorchenv/lib/python3.9/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models wei

Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 